In [18]:
import torch
import torch.nn as nn
import pickle
from transformers import AutoModel, PreTrainedTokenizerFast, AutoTokenizer
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
batch_size = 16
lr = 1e-3
epoch = 10
device = "cuda"

In [15]:
with open("./comments.bin", "rb") as f:
    data = pickle.load(f)

n = int(len(data) * 0.8)
train_ds = data[:n].values.tolist()
val_ds = data[n:].values.tolist()
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
next(iter(train_dl))

[(' 特别感动，我为什么不用这票钱去吃东西',
  ' 画面美好 意境留足 孤独感萦绕 配音不出戏 很好',
  ' 卧槽，复活快银好么？！彩蛋是灭霸',
  ' 宿命轮回，有因必有果 画风很美，后期略有些仓促，值得推荐一看 为中国传统文化打五星',
  '  好看！美队身材真好😂钢铁侠感觉像个逗比，雷神一出现我还以为布拉德皮特-_-#黑寡妇也是很sexy，最后几位大神团结在一块杀妖魔鬼怪的时候蛮感动的 最后想说IMAX效果好棒，老美这大片拍的已然无法超越😐',
  ' 改编的还是照样喜欢！喜欢就没差咯！漫威大法好好好！旺达好美好美！',
  ' 什么乱七八糟的内容？畸形的价值观，自私的女主角，为了所谓的报恩连累整个村，三角恋的恋情设计得一塌糊涂，台词让我数次起尴尬症，有模仿千与千寻的痕迹，可连它的百分之一都比不上，给一星是因为无法打零分！',
  ' 配乐是日本人做的，原画是中传学生画的，后期是外包给韩国人的，做了十二年拿着众筹的钱去旅游是主办方做的，宣传是打着中国版千与千寻的噱头卖情怀的，最后说一句，剧情是玛丽苏的，我还能说些什么？',
  ' 真的没想到，这个片子这么烂，这么脑残，太拉低老外的智商了吧 ',
  ' 重要的不是电影而是陪在你身边的人 坐第一排仰着脖子看也可以那么舒心',
  ' 大鱼一生黑 粉丝行为，电影买单 ',
  ' 画外音，不错哦，对白有点幼稚，其他都蛮好！',
  ' 漫威的忠实观众，从大一开始看他们的漫画和电影！挺不错的',
  ' 玛丽苏苏的我一脸血，感想就不说了，因为实在无话可说，我尴尬癌犯了让我歇会儿',
  ' “我们一定会再相遇的，相信我”“相信我，跟我一起跳，相信我”“我会化作人间的风雨，陪着你……”谁说这是狗血三角恋我打死谁……到最后简直哭成狗……这明明就是一个纯到不行的爱、勇气与自我实现的故事 ',
  ' 一个绿茶婊的故事 悲催的男二号 '),
 tensor([0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0])]

In [4]:
class BertClassifier(nn.Module):

    def __init__(self, pretrained_model_path, num_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_path)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # 特征提取 使用正常lr = 1e-3
        # 全量微调 使用较小lr = le-5 ~ 1e-4
        with torch.no_grad():
            # bert
            result = self.bert(input_ids, attention_mask, token_type_ids)
        # classifier
        output = self.classifier(result.pooler_output)
        return output

In [8]:
pretrained_model_path = "./pretrained_models/roberta_dianping"
tokenizer = AutoTokenizer.from_pretrained("./pretrained_models/roberta_dianping")
model = BertClassifier(pretrained_model_path, 2)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
class SummaryWarpper:
    def __init__(self):
        self.writer = SummaryWriter("logs")
        self.train_cnt = 0
        self.val_cnt = 0

    def train_loss(self, func):
        
        def warpper(loss_fn, logits, y):
            loss = func(loss_fn, logits, y)
            self.writer.add_scalar("Train Loss", loss, self.train_cnt)
            self.train_cnt += 1
            return loss
            pass
        return warpper

In [28]:
sw_loss = SummaryWarpper()
@sw_loss.train_loss
def train_loss(loss_fn, logits, y):
    loss = loss_fn(logits, y)
    return loss

def train(model, tokenizer, loss_fn, optimizer, train_data, val_data, epoch, device):
    model = model.to(device)
    print(device)
    for e in range(epoch):
        bar = tqdm(train_data)
        for X, y in bar:
            X = tokenizer(X, return_tensors="pt", padding=True)
            X, y = X.to(device), y.to(device)
            logits = model(**X)
            loss = train_loss(loss_fn, logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            bar.set_description(f"epoch:{e + 1}, train loss:{loss.item():.4f}")
    

In [30]:
train(model, tokenizer, loss_fn, optimizer, train_dl, val_dl, epoch, device)

cuda


epoch:10, train loss:0.1859: 100%|██████████| 1000/1000 [00:32<00:00, 30.85it/s]


In [35]:
def predict(model, input, device):
    model = model.to(device)
    X = tokenizer(input, return_tensors="pt", padding=True).to(device)
    logits = model(**X)
    y_hat = logits.argmax(-1)
    print(y_hat)

In [38]:
predict(model, "这里的货质量真的很好。", device)

tensor([1], device='cuda:0')
